In [ ]:
import os
import pandas as pd
import datetime
import re

def list_chat_files(date_directory):
    chat_files = []
    # Iterate through each 'Date' directory
    for date_folder in os.listdir(date_directory):
        date_path = os.path.join(date_directory, date_folder)
        if os.path.isdir(date_path):
            # Iterate through each 'Team' subdirectory
            for team_folder in os.listdir(date_path):
                team_path = os.path.join(date_path, team_folder)
                if os.path.isdir(team_path):
                    # Iterate through each 'Person' subdirectory
                    for person_folder in os.listdir(team_path):
                        person_path = os.path.join(team_path, person_folder)
                        if os.path.isdir(person_path):
                            # Process each chat file in the 'Person' subdirectory
                            for file in os.listdir(person_path):
                                if file.endswith('.txt'):  # Assuming chat files are .txt files
                                    chat_files.append(os.path.join(person_path, file))
    return chat_files

# Replace this with your actual directory path
date_directory = "C:\\Users\\mauriceyeng\\Python\\Daily-Reports\\Test\\V1_maurice\\TestingData"
chat_files = list_chat_files(date_directory)
print(chat_files)





In [ ]:
def parse_chat_file(file_path):
    chat_data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # Handle different types of lines
            # Match lines with sender and message
            message_match = re.match(r'(\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2} [ap]m) - (.*?): (.*)', line)
            # Match system messages like "You were added"
            system_match = re.match(r'(\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2} [ap]m) - (.*)', line)

            if message_match:
                date_time_str, sender, message = message_match.groups()
            elif system_match:
                date_time_str, info = system_match.groups()
                sender = None  # System messages don't have a sender
            else:
                continue  # Skip lines that don't match the pattern

            date_time = datetime.datetime.strptime(date_time_str, '%d/%m/%y, %I:%M %p')
            is_person = sender.isalpha() if sender else False
            chat_data.append((date_time, sender, is_person))

    return chat_data

# Test the function with one file
sample_chat_file = chat_files[0]  # Replace with the path of a sample chat file
parsed_data = parse_chat_file(sample_chat_file)
print(parsed_data[:5])  # Print first 5 entries for testing

In [1]:
import os
import pandas as pd
import datetime
import re

# Existing functions: list_chat_files, parse_chat_file, create_template_dataframe

# Modified populate_dataframe function
def populate_dataframe(df, parsed_data, start_column_index):
    person_column_index = start_column_index
    for entry in parsed_data:
        date_time, sender, is_person = entry
        interval_index = min((date_time.hour * 60 + date_time.minute) // 5, 279)
        interval = df.index[interval_index]

        # Initialize columns if they don't exist
        if person_column_index not in df.columns:
            df[person_column_index] = 0
        if person_column_index + 1 not in df.columns:
            df[person_column_index + 1] = 0

        # Update the dataframe
        if is_person:
            df.at[interval, person_column_index] = 1
        else:
            df.at[interval, person_column_index + 1] = 1

    # Return the next starting column index for the next chat file
    return person_column_index + 2

# New function to process all chats for each person
def process_person_chats(chat_files):
    dataframes = {}
    for file in chat_files:
        # Extract person and date information from the file path
        parts = file.split(os.sep)
        date, person = parts[-3], parts[-2]
        key = f"{date}_{person}"

        if key not in dataframes:
            dataframes[key] = create_template_dataframe()
            start_column_index = 0
        else:
            start_column_index = max(dataframes[key].columns) + 1

        parsed_data = parse_chat_file(file)
        next_column_index = populate_dataframe(dataframes[key], parsed_data, start_column_index)
    
    return dataframes

# Main script
date_directory = "C:\\Users\\mauriceyeng\\Python\\Daily-Reports\\Test\\V1_maurice\\TestingData"
chat_files = list_chat_files(date_directory)
person_dataframes = process_person_chats(chat_files)

# Example to print or save the dataframes
for key, df in person_dataframes.items():
    print(f"DataFrame for {key}")
    print(df.head())  # Print the first few rows for each dataframe
    csv_file_path = f"csvs/{key}.csv"  # Saving in the current working directory
    df.to_csv(csv_file_path)
    print(f"Saved DataFrame to {csv_file_path}")


NameError: name 'list_chat_files' is not defined

In [ ]:
template_df.to_csv("table4.csv",index=False)

In [7]:
import os
import pandas as pd
import datetime
import re

# Function to list all chat files in the directory structure
def list_chat_files(date_directory):
    chat_files = []
    for date_folder in os.listdir(date_directory):
        date_path = os.path.join(date_directory, date_folder)
        if os.path.isdir(date_path):
            for team_folder in os.listdir(date_path):
                team_path = os.path.join(date_path, team_folder)
                if os.path.isdir(team_path):
                    for person_folder in os.listdir(team_path):
                        person_path = os.path.join(team_path, person_folder)
                        if os.path.isdir(person_path):
                            for file in os.listdir(person_path):
                                if file.endswith('.txt'):
                                    chat_files.append(os.path.join(person_path, file))
    return chat_files

# Function to parse a single chat file
def parse_chat_file(file_path, expected_date):
    chat_data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            message_match = re.match(r'(\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2} [ap]m) - (.*?): (.*)', line)
            system_match = re.match(r'(\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2} [ap]m) - (.*)', line)
            if message_match:
                date_time_str, sender, message = message_match.groups()
            elif system_match:
                date_time_str, info = system_match.groups()
                sender = None
            else:
                continue

            date_time = pd.to_datetime(date_time_str, format='%d/%m/%y, %I:%M %p')

            if date_time.date() != expected_date:
                continue

            is_person = sender.isalpha() if sender else False
            chat_data.append((date_time, sender, is_person))
    return chat_data

# Function to create a template dataframe
def create_template_dataframe():
    times = [datetime.datetime(2000, 1, 1, 0, 0) + datetime.timedelta(minutes=5 * i) for i in range(280)]
    intervals = [time.strftime('%I:%M %p') for time in times]
    df = pd.DataFrame(index=intervals)
    return df

# Function to populate the dataframe
def populate_dataframe(df, parsed_data, start_column_index):
    person_column_index = start_column_index
    for entry in parsed_data:
        date_time, sender, is_person = entry
        interval_index = min((date_time.hour * 60 + date_time.minute) // 5, 279)
        interval = df.index[interval_index]

        if person_column_index not in df.columns:
            df[person_column_index] = 0
        if person_column_index + 1 not in df.columns:
            df[person_column_index + 1] = 0

        if is_person:
            df.at[interval, person_column_index] = 1
        else:
            df.at[interval, person_column_index + 1] = 1

    return person_column_index + 2

def process_person_chats(chat_files):
    dataframes = {}
    for file in chat_files:
        parts = file.split(os.sep)
        date_folder, person = parts[-4], parts[-2]

        try:
            expected_date = pd.to_datetime(date_folder).date()
        except ValueError:
            print(f"Skipping file due to incorrect date format in folder name: {file}")
            continue

        key = f"{expected_date.strftime('%Y-%m-%d')}_{person}"

        if key not in dataframes:
            dataframes[key] = create_template_dataframe()
            start_column_index = 0
        else:
            # Check if the dataframe already has columns
            if not dataframes[key].columns.empty:
                start_column_index = max(dataframes[key].columns) + 1
            else:
                start_column_index = 0

        parsed_data = parse_chat_file(file, expected_date)
        next_column_index = populate_dataframe(dataframes[key], parsed_data, start_column_index)
    
    return dataframes




# Main script
date_directory = "C:\\Users\\mauriceyeng\\Python\\Daily-Reports\\Test\\V1_maurice\\TestingData"
chat_files = list_chat_files(date_directory)
person_dataframes = process_person_chats(chat_files)

# Save each dataframe as a CSV file in the current working directory
for key, df in person_dataframes.items():
    csv_file_path = f"{key}.csv"
    df.to_csv(csv_file_path)
    print(f"Saved DataFrame to {csv_file_path}")


ValueError: max() arg is an empty sequence

In [6]:
import pandas as pd
import os

# Sample file path from your error message
sample_file_path = "C:\\Users\\mauriceyeng\\Python\\Daily-Reports\\Test\\V1_maurice\\TestingData\\2023-11-18\\EWYL TEAM\\Ananya_Edoofa\\WhatsApp Chat with Twazanga EWYL22E1057.txt"

# Extracting the date part from the file path
parts = sample_file_path.split(os.sep)
date_folder = parts[-4]  # Adjust index based on your folder structure

# Try parsing the date
try:
    expected_date = pd.to_datetime(date_folder).date()
    print(f"Parsed Date: {expected_date}")
except ValueError as e:
    print(f"Error parsing date: {e}")


Parsed Date: 2023-11-18
